In [74]:
import json
import logging
import re
import folium

from django.contrib.gis.geos.polygon import Polygon

from madmex.api.remote import UsgsApi, EspaApi
from madmex.management.base import AntaresBaseCommand
from madmex.models import Country, Footprint, Region, Order

In [75]:
usgs_client = UsgsApi()
usgs_client.login()

https://earthexplorer.usgs.gov/inventory/json/v/stable/login
Successfully logged in to earth explorer USGS api.


True

In [222]:
start_date = '2001-01-01' 
end_date = '2002-12-31'
landsat = 5
shape_name = 22049
#shape_name = 'Jalisco'
cloud_cover = 10

In [223]:
if isinstance(shape_name, int):
    print("Es dígito")
    shape_object = Footprint.objects.get(name=shape_name)
elif isinstance(shape_name, str):
    try:
        shape_object = Country.objects.get(name=shape_name)
        print('Country %s was loaded.' % shape_name)
    except:
        try:
            shape_object = Region.objects.get(name=shape_name)
            print('Region %s was loaded.' % shape_name)
        except:
            shape_object = None
    print("Es región")

Es dígito


In [224]:
espa_client = EspaApi()

In [225]:
tile = json.loads(shape_object.the_geom.geojson)
m = folium.Map(location=[24, -102],
               tiles='OpenStreetMap',
               zoom_start=5.5)
folium.GeoJson(tile).add_to(m)
m

In [226]:
extent = shape_object.the_geom.extent

In [227]:
# Landsat 5
collection_usgs = 'LANDSAT_TM_C1'
collection_espa = 'tm5_collection'
collection_regex = '^lt05_{1}\\w{4}_{1}[0-9]{6}_{1}[0-9]{8}_{1}[0-9]{8}_{1}[0-9]{2}_{1}\\w{2}$'

In [228]:
data = usgs_client.search(extent, collection_usgs, start_date=start_date, end_date=end_date, max_cloud_cover=cloud_cover).get('data')

https://earthexplorer.usgs.gov/inventory/json/v/stable/search


In [229]:
def point_from_object(coords):
    return (coords.get('longitude'), coords.get('latitude'))

In [230]:
products = ['sr', 'pixel_qa']
interest = []
if data:
    results= data.get('results')
    if results:
        for scene in results:
            coords = tuple(point_from_object(scene.get(coord)) for coord in ['lowerLeftCoordinate', 'upperLeftCoordinate', 'upperRightCoordinate', 'lowerRightCoordinate', 'lowerLeftCoordinate'])
            scene_extent = Polygon(coords)
            entity_id = scene.get('displayId')
            print(entity_id)
            #print(entity_id)
            # we use the same regular expression that espa uses to filter the names that are valid; otherwise, the order throws an error
            if isinstance(shape_name, int): 
                if str(shape_name) in entity_id:
                    #if scene_extent.intersects(shape_object.the_geom) and re.match(collection_regex, entity_id):
                    interest.append(entity_id)
            elif scene_extent.intersects(shape_object.the_geom):# and re.match(collection_regex, entity_id):
                interest.append(entity_id)
                #footprint, _ = Footprint.objects.get_or_create(name=entity_id,the_geom=scene_extent)
print(json.dumps(interest, indent=4))

LT05_L1TP_023049_20010106_20161212_01_T1
LT05_L1TP_022049_20010131_20161212_01_T1
LT05_L1TP_023048_20010311_20161212_01_T1
LT05_L1TP_023049_20010311_20161212_01_T1
LT05_L1TP_021049_20010313_20161212_01_T1
LT05_L1TP_021050_20010313_20161212_01_T1
LT05_L1TP_023049_20010327_20161211_01_T1
LT05_L1TP_021048_20010329_20161211_01_T1
LT05_L1TP_021049_20010329_20161211_01_T1
LT05_L1TP_022048_20010405_20161211_01_T1
LT05_L1TP_022049_20010405_20161211_01_T1
LT05_L1TP_023049_20010428_20161211_01_T1
LT05_L1TP_021050_20010430_20161211_01_T1
LT05_L1TP_023049_20010514_20161211_01_T1
LT05_L1TP_023048_20010615_20161211_01_T1
[
    "LT05_L1TP_022049_20010131_20161212_01_T1",
    "LT05_L1TP_022049_20010405_20161211_01_T1"
]
